```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [ ]:
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import pandas as pd
from pyspark.sql.types import StructType, StructField,StringType, LongType, IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit
from sedona.spark import *
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit
import os


# Create Spark Session for application

In [ ]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.5.0,'
           'org.datasyslab:geotools-wrapper:1.5.0-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)

sc = sedona.sparkContext


# Geotiff Loader 

1. Loader takes as input a path to directory which contains geotiff files or a path to particular geotiff file
2. Loader will read geotiff image in a struct named image which contains multiple fields as shown in the schema below which can be extracted using spark SQL

In [ ]:
# Path to directory of geotiff images 
DATA_DIR = "./data/raster/"

In [ ]:
df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").option("disableErrorInCRS", False).load(DATA_DIR)
df.printSchema()

In [ ]:
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")
df.show(5)

# Extract a particular band from geotiff dataframe using RS_GetBand()


In [ ]:
'''RS_GetBand() will fetch a particular band from given data array which is the concatenation of all the bands'''

df = df.selectExpr("Geom","RS_GetBand(data, 1,bands) as Band1","RS_GetBand(data, 2,bands) as Band2","RS_GetBand(data, 3,bands) as Band3", "RS_GetBand(data, 4,bands) as Band4")
df.createOrReplaceTempView("allbands")
df.show(5)

# Map Algebra operations on band values

In [ ]:
'''RS_NormalizedDifference can be used to calculate NDVI for a particular geotiff image since it uses same computational formula as ndvi'''

NomalizedDifference = df.selectExpr("RS_NormalizedDifference(Band1, Band2) as normDiff")
NomalizedDifference.show(5)

In [ ]:
'''RS_Mean() can used to calculate mean of piel values in a particular spatial band'''
meanDF = df.selectExpr("RS_Mean(Band1) as mean")
meanDF.show(5)

In [ ]:
""" RS_Mode() is used to calculate mode in an array of pixels and returns a array of double with size 1 in case of unique mode"""
modeDF = df.selectExpr("RS_Mode(Band1) as mode")
modeDF.show(5)

In [ ]:
'''RS_GreaterThan() is used to mask all the values with 1 which are greater than a particular threshold'''
greaterthanDF = sedona.sql("Select RS_GreaterThan(Band1,1000.0) as greaterthan from allbands")
greaterthanDF.show()

In [ ]:
'''RS_GreaterThanEqual() is used to mask all the values with 1 which are greater than a particular threshold'''

greaterthanEqualDF = sedona.sql("Select RS_GreaterThanEqual(Band1,360.0) as greaterthanEqual from allbands")
greaterthanEqualDF.show()

In [ ]:
'''RS_LessThan() is used to mask all the values with 1 which are less than a particular threshold'''
lessthanDF = sedona.sql("Select RS_LessThan(Band1,1000.0) as lessthan from allbands")
lessthanDF.show()

In [ ]:
'''RS_LessThanEqual() is used to mask all the values with 1 which are less than equal to a particular threshold'''
lessthanEqualDF = sedona.sql("Select RS_LessThanEqual(Band1,2890.0) as lessthanequal from allbands")
lessthanEqualDF.show()

In [ ]:
'''RS_Add() can add two spatial bands together'''
sumDF = df.selectExpr("RS_Add(Band1, Band2) as sumOfBand")
sumDF.show(5)

In [ ]:
'''RS_Subtract() can subtract two spatial bands together'''
subtractDF = df.selectExpr("RS_Subtract(Band1, Band2) as diffOfBand")
subtractDF.show(5)

In [ ]:
'''RS_Multiply() can multiple two bands together'''
multiplyDF = df.selectExpr("RS_Multiply(Band1, Band2) as productOfBand")
multiplyDF.show(5)

In [ ]:
'''RS_Divide() can divide two bands together'''
divideDF = df.selectExpr("RS_Divide(Band1, Band2) as divisionOfBand")
divideDF.show(5)

In [ ]:
'''RS_MultiplyFactor() will multiply a factor to a spatial band'''
mulfacDF = df.selectExpr("RS_MultiplyFactor(Band2, 2) as target")
mulfacDF.show(5)

In [ ]:
'''RS_BitwiseAND() will return AND between two values of Bands'''
bitwiseAND = df.selectExpr("RS_BitwiseAND(Band1, Band2) as AND")
bitwiseAND.show(5)

In [ ]:
'''RS_BitwiseOR() will return OR between two values of Bands'''
bitwiseOR = df.selectExpr("RS_BitwiseOR(Band1, Band2) as OR")
bitwiseOR.show(5)

In [ ]:
'''RS_Count() will calculate the total number of occurrence of a target value'''
countDF = df.selectExpr("RS_Count(RS_GreaterThan(Band1,1000.0), 1.0) as count")
countDF.show(5)

In [ ]:
'''RS_Modulo() will calculate the modulus of band value with respect to a given number'''
moduloDF = df.selectExpr("RS_Modulo(Band1, 21.0) as modulo ")
moduloDF.show(5)

In [ ]:
'''RS_SquareRoot() will calculate calculate square root of all the band values up to two decimal places'''
rootDF = df.selectExpr("RS_SquareRoot(Band1) as root")
rootDF.show(5)


In [ ]:
'''RS_LogicalDifference() will return value from band1 if value at that particular location is not equal tp band1 else it will return 0'''
logDiff = df.selectExpr("RS_LogicalDifference(Band1, Band2) as loggDifference")
logDiff.show(5)

In [ ]:
'''RS_LogicalOver() will iterate over two bands and return value of first band if it is not equal to 0 else it will return value from later band'''
logOver = df.selectExpr("RS_LogicalOver(Band3, Band2) as logicalOver")
logOver.show(5)

# Visualising Geotiff Images

1. Normalize the bands in range [0-255] if values are greater than 255
2. Process image using RS_Base64() which converts in into a base64 string
3. Embed results of RS_Base64() in RS_HTML() to embed into IPython notebook
4. Process results of RS_HTML() as below:

In [ ]:
'''Plotting images as a dataframe using geotiff Dataframe.'''

df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").load(DATA_DIR)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")

df = df.selectExpr("RS_GetBand(data,1,bands) as targetband", "height", "width", "bands", "Geom")
df_base64 = df.selectExpr("Geom", "RS_Base64(height,width,RS_Normalize(targetBand), RS_Array(height*width,0.0), RS_Array(height*width, 0.0)) as red","RS_Base64(height,width,RS_Array(height*width, 0.0), RS_Normalize(targetBand), RS_Array(height*width, 0.0)) as green", "RS_Base64(height,width,RS_Array(height*width, 0.0),  RS_Array(height*width, 0.0), RS_Normalize(targetBand)) as blue","RS_Base64(height,width,RS_Normalize(targetBand), RS_Normalize(targetBand),RS_Normalize(targetBand)) as RGB" )
df_HTML = df_base64.selectExpr("Geom","RS_HTML(red) as RedBand","RS_HTML(blue) as BlueBand","RS_HTML(green) as GreenBand", "RS_HTML(RGB) as CombinedBand")
df_HTML.show(5)

In [ ]:
display(HTML(df_HTML.limit(2).toPandas().to_html(escape=False)))

# Writing GeoTiff Images

In [ ]:
'''Writing GeoTiff DataFrames as GeoTiff Images'''

df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").load(DATA_DIR)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")

SAVE_PATH = "./data/raster-written/"
df.write.mode("overwrite").format("geotiff").option("writeToCRS", "EPSG:4326").option("fieldGeometry", "Geom").option("fieldNBands", "bands").save(SAVE_PATH)

In [ ]:
'''Writing GeoTiff Images in a Single Partition'''
df.coalesce(1).write.mode("overwrite").format("geotiff").option("writeToCRS", "EPSG:4326").option("fieldGeometry", "Geom").option("fieldNBands", "bands").save(SAVE_PATH)

In [ ]:
'''Find the Partition of the Written GeoTiff Images.
   If you did not write with coalesce(1), change the below code to adjust the writtenPath'''
writtenPath = SAVE_PATH
dirList = os.listdir(writtenPath)
for item in dirList:
    if os.path.isdir(writtenPath + "/" + item):
        writtenPath += "/" + item
        break

In [ ]:
'''Load and Visualize Written GeoTiff Image.'''

df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").load(writtenPath)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")

df = df.selectExpr("RS_GetBand(data,1,bands) as targetband", "height", "width", "bands", "Geom")
df_base64 = df.selectExpr("Geom", "RS_Base64(height,width,RS_Normalize(targetBand), RS_Array(height*width,0.0), RS_Array(height*width, 0.0)) as red","RS_Base64(height,width,RS_Array(height*width, 0.0), RS_Normalize(targetBand), RS_Array(height*width, 0.0)) as green", "RS_Base64(height,width,RS_Array(height*width, 0.0),  RS_Array(height*width, 0.0), RS_Normalize(targetBand)) as blue","RS_Base64(height,width,RS_Normalize(targetBand), RS_Normalize(targetBand),RS_Normalize(targetBand)) as RGB" )
df_HTML = df_base64.selectExpr("Geom","RS_HTML(red) as RedBand","RS_HTML(blue) as BlueBand","RS_HTML(green) as GreenBand", "RS_HTML(RGB) as CombinedBand")
display(HTML(df_HTML.limit(2).toPandas().to_html(escape=False)))

# Transformation of GeoTiff Images

In [ ]:
'''First load GeoTiff Images'''
df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").option("disableErrorInCRS", False).load(DATA_DIR)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")
df.show(5)

In [ ]:
# First extract the bands for which normalized difference index needs to be calculated
df = df.selectExpr("origin", "geom", "width", "height", "data", "bands", "RS_GetBand(data, 1, bands) as band1", "RS_GetBand(data, 2, bands) as band2")
# Get the normalized difference index between the extracted bands
df = df.selectExpr("origin", "geom", "width", "height", "data", "bands", "RS_NormalizedDifference(band2, band1) as normalizedDifference")
df.show(5)

In [ ]:
'''RS_Append() takes the data array containing bands, a new band to be appended, and number of total bands in the data array.
    It appends the new band to the end of the data array and returns the appended data'''

df = df.selectExpr("origin", "geom", "RS_Append(data, normalizedDifference, bands) as data_edited", "height", "width", "bands").drop("data")
df = df.withColumn("nBand_edited", col("bands") + 1).drop("bands")
df.show()

In [ ]:
'''Writing GeoTiff DataFrames as GeoTiff Images'''
SAVE_PATH = "./data/raster-written/"
df.coalesce(1).write.mode("overwrite").format("geotiff").option("writeToCRS", "EPSG:4326").option("fieldGeometry", "geom").option("fieldNBands", "nBand_edited").option("fieldData", "data_edited").save(SAVE_PATH)

# User can also create some UDF manually to manipulate Geotiff dataframes

In [ ]:
'''Sample UDF calculates sum of all the values in a band which are greater than 1000.0'''

def SumOfValues(band):
    total = 0.0
    for num in band:
        if num>1000.0:
            total+=1
    return total

df = sedona.read.format("geotiff").option("dropInvalid",True).option("readToCRS", "EPSG:4326").load(DATA_DIR)
df = df.selectExpr("image.origin as origin","ST_GeomFromWkt(image.geometry) as Geom", "image.height as height", "image.width as width", "image.data as data", "image.nBands as bands")
df = df.selectExpr("RS_GetBand(data,1,bands) as targetband", "height", "width", "bands", "Geom")
    
calculateSum = udf(SumOfValues, DoubleType())
sedona.udf.register("RS_Sum", calculateSum)

sumDF = df.selectExpr("RS_Sum(targetband) as sum")
sumDF.show()

In [ ]:
'''Sample UDF to visualize a particular region of a GeoTiff image'''

def generatemask(band, width,height):
    for (i,val) in enumerate(band):
        if (i%width>=12 and i%width<26) and (i%height>=12 and i%height<26):
            band[i] = 255.0
        else:
            band[i] = 0.0
    return band

maskValues = udf(generatemask, ArrayType(DoubleType()))
sedona.udf.register("RS_MaskValues", maskValues)


df_base64 = df.selectExpr("Geom", "RS_Base64(height,width,RS_Normalize(targetband), RS_Array(height*width,0.0), RS_Array(height*width, 0.0), RS_MaskValues(targetband,width,height)) as region" )
df_HTML = df_base64.selectExpr("Geom","RS_HTML(region) as selectedregion")
display(HTML(df_HTML.limit(2).toPandas().to_html(escape=False)))
